# Spark Apache (семинары)
## Урок 5. Spark on scala
**Условие:** создайте csv файл с таким содержимым:

title,author,genre,sales,year  
"1984", "George Orwell", "Science Fiction", 5000, 1949  
"The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954  
"To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960  
"The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951  
"The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925  

**Задание:**
* Используя Spark прочитайте данные из файла csv.
* Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.
* Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.
* Отсортируйте данные по общему объему продаж в порядке убывания.
* Выведите результаты на экран.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
import pyspark.sql.functions as FN

In [2]:
# Инициализация Spark сессии
spark = SparkSession.builder \
    .appName("SalesOfBooks") \
    .getOrCreate()

In [3]:
# Схема данных
schema = StructType([
    StructField("title", StringType(), True),
    StructField("author", StringType(), True),
    StructField("genre", StringType(), True),
    StructField("sales", DoubleType(), True),
    StructField("year", IntegerType(), True) 
])

In [4]:
# Вариант 1a. Читаем данные из CSV файла по схеме
books_df = spark.read \
    .option("header", "true") \
    .option("delimiter", ",") \
    .option("nullValue", "") \
    .schema(schema) \
    .csv("books.csv")

print("Продажи книг")
books_df.show()

Продажи книг
+--------------------+-------------------+--------------------+-------+----+
|               title|             author|               genre|  sales|year|
+--------------------+-------------------+--------------------+-------+----+
|     It Ends with Us|     Colleen Hoover|             Romance|24500.0|2016|
|  The Silent Patient|   Alex Michaelides|Psychological Thr...|18750.0|2019|
|The Seven Husband...|Taylor Jenkins Reid|  Historical Fiction|16500.0|2017|
|         Fourth Wing|     Rebecca Yarros|             Fantasy|15800.0|2023|
|          Iron Flame|     Rebecca Yarros|             Fantasy|15200.0|2023|
|The Last Thing He...|         Laura Dave|             Mystery|14200.0|2021|
|The Midnight Library|          Matt Haig|Contemporary Fiction|13800.0|2020|
|Lessons in Chemistry|      Bonnie Garmus|  Historical Fiction|13500.0|2022|
|The Heaven & Eart...|      James McBride|  Historical Fiction|12800.0|2023|
|Tomorrow, and Tom...|    Gabrielle Zevin|    Literary Fiction|

In [5]:
# Вариант 1b. Читаем данные из CSV файла с автоматическим определением схемы
books_df = spark.read.load('books.csv', format="csv",
                           sep=",", header="true", Infer_schema=True)

books_df.printSchema()

# Измменяем типы данных столбцов
books_df.withColumn("sales", books_df["sales"].cast("double")) \
        .withColumn("year", books_df["year"].cast("int"))

books_df.printSchema()

print("Продажи книг")
books_df.show()

root
 |-- title: string (nullable = true)
 |-- author: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- sales: string (nullable = true)
 |-- year: string (nullable = true)

root
 |-- title: string (nullable = true)
 |-- author: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- sales: string (nullable = true)
 |-- year: string (nullable = true)

Продажи книг
+--------------------+-------------------+--------------------+-----+----+
|               title|             author|               genre|sales|year|
+--------------------+-------------------+--------------------+-----+----+
|     It Ends with Us|     Colleen Hoover|             Romance|24500|2016|
|  The Silent Patient|   Alex Michaelides|Psychological Thr...|18750|2019|
|The Seven Husband...|Taylor Jenkins Reid|  Historical Fiction|16500|2017|
|         Fourth Wing|     Rebecca Yarros|             Fantasy|15800|2023|
|          Iron Flame|     Rebecca Yarros|             Fantasy|15200|2023|
|The 

In [6]:
# Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.
filtered_df = books_df.filter("sales > 3000.0")

print("Книги, продажи которых превышают 3000 экземпляров")
filtered_df.show()

Книги, продажи которых превышают 3000 экземпляров
+--------------------+-------------------+--------------------+-----+----+
|               title|             author|               genre|sales|year|
+--------------------+-------------------+--------------------+-----+----+
|     It Ends with Us|     Colleen Hoover|             Romance|24500|2016|
|  The Silent Patient|   Alex Michaelides|Psychological Thr...|18750|2019|
|The Seven Husband...|Taylor Jenkins Reid|  Historical Fiction|16500|2017|
|         Fourth Wing|     Rebecca Yarros|             Fantasy|15800|2023|
|          Iron Flame|     Rebecca Yarros|             Fantasy|15200|2023|
|The Last Thing He...|         Laura Dave|             Mystery|14200|2021|
|The Midnight Library|          Matt Haig|Contemporary Fiction|13800|2020|
|Lessons in Chemistry|      Bonnie Garmus|  Historical Fiction|13500|2022|
|The Heaven & Eart...|      James McBride|  Historical Fiction|12800|2023|
|Tomorrow, and Tom...|    Gabrielle Zevin|    Lite

In [7]:
# Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.
agregated_df = books_df.groupBy("genre").agg(
    FN.sum("sales").alias("sum_sales_by_genre"))

print("Общий объем продаж для каждого жанра")
agregated_df.show()

Общий объем продаж для каждого жанра
+--------------------+------------------+
|               genre|sum_sales_by_genre|
+--------------------+------------------+
|        Cozy Mystery|            9800.0|
|             Romance|           30400.0|
|       Short Stories|            1050.0|
|            Thriller|           22900.0|
|  Historical Romance|            2800.0|
|           Adventure|            2000.0|
|     Southern Gothic|            2400.0|
|Contemporary Fiction|           13800.0|
|   Self-Help Fiction|           16400.0|
|      Memoir Fiction|           10200.0|
|             Fantasy|          161455.0|
|        Dark Fantasy|            6600.0|
|    Post-Apocalyptic|            2850.0|
|Philosophical Fic...|            2600.0|
|         Young Adult|            2900.0|
|    Literary Fiction|           47200.0|
|Supernatural Thri...|            1750.0|
|             Mystery|           39450.0|
|Supernatural Fiction|            3300.0|
|        Crime Horror|            1600.

In [8]:
# Отсортируйте данные по общему объему продаж в порядке убывания.
sorted_df = books_df.sort("sales", ascending=False)

print("Объем продаж книг по убыванию")
sorted_df.show()

Объем продаж книг по убыванию
+--------------------+-------------------+--------------------+-----+----+
|               title|             author|               genre|sales|year|
+--------------------+-------------------+--------------------+-----+----+
|The Thursday Murd...|      Richard Osman|        Cozy Mystery| 9800|2020|
|   Klara and the Sun|     Kazuo Ishiguro|     Science Fiction| 9500|2021|
|    Wizard and Glass|       Stephen King|        Dark Fantasy|  950|1997|
|The Last Argument...|    Joe Abercrombie|             Fantasy|  950|2008|
|       Malibu Rising|Taylor Jenkins Reid|  Historical Fiction| 9200|2021|
| Wolves of the Calla|       Stephen King|        Dark Fantasy|  900|2003|
|    Best Served Cold|    Joe Abercrombie|             Fantasy|  900|2009|
|   The Tyrant's Tomb|       Rick Riordan|             Fantasy|  900|2019|
|            The Push|     Ashley Audrain|Psychological Thr...| 8900|2021|
|      The Sanatorium|       Sarah Pearse|            Thriller| 8600|2

In [10]:
# Применяем все обработки к исходному DataFrame
print("Общий объём продаж книг по жанрам с количеством более 3000 экземпляров")
result_df = books_df.filter("sales > 3000.0") \
                    .groupBy("genre") \
                    .agg(FN.sum("sales").alias("sum_sales_by_genre")) \
                    .sort("sum_sales_by_genre", ascending=False) \
                    .show()

Общий объём продаж книг по жанрам с количеством более 3000 экземпляров
+--------------------+------------------+
|               genre|sum_sales_by_genre|
+--------------------+------------------+
|  Historical Fiction|          118900.0|
|             Fantasy|           67400.0|
|    Literary Fiction|           45000.0|
|Psychological Thr...|           37650.0|
|             Mystery|           33700.0|
|             Romance|           28300.0|
|     Science Fiction|           23900.0|
|            Thriller|           18400.0|
|      Legal Thriller|           18400.0|
|   Self-Help Fiction|           16400.0|
|Contemporary Fiction|           13800.0|
|   Dystopian Fiction|           10600.0|
|      Memoir Fiction|           10200.0|
|        Cozy Mystery|            9800.0|
|      Erotic Romance|            3700.0|
|  Paranormal Romance|            3500.0|
+--------------------+------------------+



In [11]:
spark.stop()